<a href="https://colab.research.google.com/github/Parinita-Jain/Course_Handouts_NLP_using_DL/blob/main/SpamClassificationFeatureEngineeringTfidfGlove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download("all")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
nlp=spacy.load("en_core_web_sm")
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [121]:
df=pd.read_csv("/content/spam.csv")
df.head()

,Category,Message
0,not spam,"Go until jurong point, crazy.. Available only ..."
1,not spam,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,not spam,U dun say so early hor... U c already then say...
4,not spam,"Nah I don't think he goes to usf, he lives aro..."


In [122]:
df["Category"].value_counts(normalize=True)

not spam    0.865937
spam        0.134063
Name: Category, dtype: float64

In [123]:
stop_words=set(stopwords.words("english"))
lemmatizer=WordNetLemmatizer()
def preprocessing(text):
  text=text.lower()
  text=re.sub("[^a-z ]+"," ",text)
  text=re.sub("[\s]+"," ",text)
  token=[lemmatizer.lemmatize(i) for i in text.split() if i not in stop_words]

  return " ".join(token)


In [124]:
df["Cleaned"]=df["Message"].apply(preprocessing)
df.head()

,Category,Message,Cleaned
0,not spam,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,not spam,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,not spam,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,not spam,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though


In [125]:
len(df["Message"][0])

111

In [126]:
len(df["Cleaned"][0])

82

In [127]:
df["word_count"]=df["Message"].apply(lambda x:len(x.split()))
df["word_count_cleand"]=df["Cleaned"].apply(lambda x:len(x.split()))
df["char_count"]=df["Cleaned"].apply(lambda x:len(x))
df["char_count_without_spaces"]=df["Cleaned"].apply(lambda x:len(x.replace(" ","")))

In [128]:
df.head()

,Category,Message,Cleaned,word_count,word_count_cleand,char_count,char_count_without_spaces
0,not spam,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,20,16,82,67
1,not spam,Ok lar... Joking wif u oni...,ok lar joking wif u oni,6,6,23,18
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...,28,21,101,81
3,not spam,U dun say so early hor... U c already then say...,u dun say early hor u c already say,11,9,35,27
4,not spam,"Nah I don't think he goes to usf, he lives aro...",nah think go usf life around though,13,7,35,29


In [129]:
from nltk.tag import pos_tag

In [130]:
# Dictionary of noun and verb POS tags
pos_dic = {"noun" : ["NNP", "NN", "NNS", "NNPS"], "verb" : ["VBZ", "VB", "VBD","VBG", "VBN","VBP"]}

def pos(text,family):
  count=0
  pos=pos_tag(text.split())

  for i in pos:

    if i[1] in pos_dic[family]:
      count += 1
  return count

In [131]:
# Applying the function
df["noun_count"] = df["Cleaned"].apply(lambda x : pos(x, "noun"))
df["verb_count"] = df["Cleaned"].apply(lambda x : pos(x, "verb"))

In [132]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["Category"]=le.fit_transform(df["Category"])

In [133]:
df.iloc[0]

Category                                                                     0
Message                      Go until jurong point, crazy.. Available only ...
Cleaned                      go jurong point crazy available bugis n great ...
word_count                                                                  20
word_count_cleand                                                           16
char_count                                                                  82
char_count_without_spaces                                                   67
noun_count                                                                   6
verb_count                                                                   3
Name: 0, dtype: object

In [134]:
train = df[['word_count', 'word_count_cleand', 'char_count',
              'char_count_without_spaces','noun_count',
              'verb_count']]

In [135]:
target=df.iloc[:,0]

In [136]:
# Train-Validation split
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train, target, random_state=20, stratify=target)

In [137]:
# Train and Validatio dataset
(x_train.shape, y_train.shape), (x_valid.shape, y_valid.shape)

(((4179, 6), (4179,)), ((1393, 6), (1393,)))

In [138]:
from sklearn import naive_bayes
model=naive_bayes.MultinomialNB()
# Fit model on training data
model.fit(x_train, y_train)
# Prediction on training data
pred_train = model.predict(x_train)
# Prediction on validation data
pred_valid = model.predict(x_valid)
# Accuracy
from sklearn.metrics import accuracy_score
# Training accuracy
accuracy_score(y_train, pred_train)
# Validation accuracy
accuracy_score(y_valid, pred_valid)


0.8693467336683417

In [139]:
# Import Tf-Idf Vectoriser
from sklearn.feature_extraction.text import TfidfVectorizer
# Tf-Idf Vectoriser
word_tfidf = TfidfVectorizer(max_features=500)

In [140]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_tfidf=TfidfVectorizer(max_features=500)
word_vectors=word_tfidf.fit_transform(df["Cleaned"]).toarray()

In [141]:
word_vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [142]:
wv=pd.DataFrame(word_vectors)
# Combining meta features and Tf-Idf features

# List of meta features
meta_features = ['word_count', 'word_count_cleand',
       'char_count', 'char_count_without_spaces', 'noun_count',
       'verb_count']

# Meta features
feature_set1 = df[meta_features]

f=pd.concat([wv,df[meta_features]],axis=1)
f

,0,1,2,3,4,5,6,7,8,9,...,496,497,498,499,word_count,word_count_cleand,char_count,char_count_without_spaces,noun_count,verb_count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20,16,82,67,6,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,6,23,18,3,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,28,21,101,81,9,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,9,35,27,3,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13,7,35,29,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30,17,83,67,9,1
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8,5,25,21,3,1
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10,3,20,18,3,0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,26,13,78,66,4,3


In [143]:
from sklearn.ensemble import RandomForestClassifier
f.columns = f.columns.astype(str)
x_train, x_valid, y_train, y_valid = train_test_split(f, target, random_state=20, stratify=target)
model=RandomForestClassifier(n_estimators=501,criterion="entropy")
model.fit(x_train,y_train)
y_pred=model.predict(x_valid)
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_valid,y_pred)
cm

array([[1200,    6],
       [  26,  161]])

In [144]:
from sklearn.metrics import classification_report
print(classification_report(y_valid,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1206
           1       0.96      0.86      0.91       187

    accuracy                           0.98      1393
   macro avg       0.97      0.93      0.95      1393
weighted avg       0.98      0.98      0.98      1393



In [145]:
import tensorflow
from tensorflow.keras.layers import Embedding,Dense,LSTM,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

In [162]:
x=df["Cleaned"]
y=df["Category"]
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=20, stratify=y)
tok=Tokenizer()
tok.fit_on_texts(x_train)
vocabulary=tok.word_index
vocab_length=len(vocabulary)
train_sequences=tok.texts_to_sequences(x_train)
doc_length=[]
for doc in train_sequences:
  doc_length.append(len(doc))
max_length=int(np.quantile(doc_length,0.99))

train_matrix=sequence.pad_sequences(train_sequences,maxlen=max_length)
test_sequences=tok.texts_to_sequences(x_test)
test_matrix=sequence.pad_sequences(test_sequences,maxlen=max_length)

model=Sequential()
model.add(Embedding(input_dim=vocab_length+1,output_dim=32,input_length=int(max_length),mask_zero=True))
model.add(Flatten())
model.add(Dense(64,activation="tanh"))
model.add(Dense(32,activation="tanh"))
model.add(Dense(16,activation="tanh"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="adam",loss="binary_crossentropy")

model.fit(train_matrix,y_train,epochs=10,batch_size=32)

ypred=model.predict(test_matrix)
ypred=np.where(y_pred>=0.5,1,0)




Epoch 1/10
131/131 [==============================] - 12s 79ms/step - loss: 0.1641
Epoch 2/10
131/131 [==============================] - 5s 38ms/step - loss: 0.0256
Epoch 3/10
131/131 [==============================] - 2s 17ms/step - loss: 0.0138
Epoch 4/10
131/131 [==============================] - 2s 19ms/step - loss: 0.0049
Epoch 5/10
131/131 [==============================] - 2s 12ms/step - loss: 0.0036
Epoch 6/10
131/131 [==============================] - 1s 8ms/step - loss: 0.0015
Epoch 7/10
131/131 [==============================] - 1s 8ms/step - loss: 7.4108e-04
Epoch 8/10
131/131 [==============================] - 1s 9ms/step - loss: 4.6761e-04
Epoch 9/10
131/131 [==============================] - 1s 11ms/step - loss: 3.7713e-04
Epoch 10/10
44/44 [==============================] - 0s 2ms/step


'              precision    recall  f1-score   support\n\n           0       0.99      1.00      0.99      1206\n           1       1.00      0.92      0.96       187\n\n    accuracy                           0.99      1393\n   macro avg       0.99      0.96      0.98      1393\nweighted avg       0.99      0.99      0.99      1393\n'

In [163]:
print(classification_report(y_test,y_pred.round()))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1206
           1       1.00      0.92      0.96       187

    accuracy                           0.99      1393
   macro avg       0.99      0.96      0.98      1393
weighted avg       0.99      0.99      0.99      1393



In [164]:
embeddings_index={}
f=open("glove.6B.100d.txt",encoding="utf-8")
for line in f:
  values=line.split()
  word=values[0]
  coefs=np.asarray(values[1:],dtype="float32")
  embeddings_index[word]=coefs
f.close()

In [166]:
embedding_dim=100
embedding_matrix=np.zeros((vocab_length+1,embedding_dim))
for word,i in tok.word_index.items():
  embedding_vector=embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector

In [168]:
from pandas.io.stata import max_len_string_array
model=Sequential()
model.add(Embedding(vocab_length+1,embedding_dim,
                     weights=[embedding_matrix],input_length=int(max_length),trainable=False))

model.add(Flatten())
model.add(Dense(64,activation="tanh"))
model.add(Dense(32,activation="tanh"))
model.add(Dense(16,activation="tanh"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="adam",loss="binary_crossentropy")

model.fit(train_matrix,y_train,epochs=10,batch_size=32)

ypred=model.predict(test_matrix)
ypred=np.where(y_pred>=0.5,1,0)



Epoch 1/10
131/131 [==============================] - 3s 6ms/step - loss: 0.1568
Epoch 2/10
131/131 [==============================] - 0s 4ms/step - loss: 0.0513
Epoch 3/10
131/131 [==============================] - 1s 5ms/step - loss: 0.0271
Epoch 4/10
131/131 [==============================] - 1s 5ms/step - loss: 0.0209
Epoch 5/10
131/131 [==============================] - 1s 5ms/step - loss: 0.0235
Epoch 6/10
131/131 [==============================] - 1s 4ms/step - loss: 0.0255
Epoch 7/10
131/131 [==============================] - 0s 3ms/step - loss: 0.0189
Epoch 8/10
131/131 [==============================] - 0s 3ms/step - loss: 0.0101
Epoch 9/10
131/131 [==============================] - 0s 3ms/step - loss: 0.0089
Epoch 10/10
44/44 [==============================] - 0s 2ms/step


In [169]:
print(classification_report(y_test,y_pred.round()))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1206
           1       1.00      0.92      0.96       187

    accuracy                           0.99      1393
   macro avg       0.99      0.96      0.98      1393
weighted avg       0.99      0.99      0.99      1393

